# Post-impact temperature files & cooling setup
### Sarah Steele, 2023


## Utilities

In [9]:
## import important things
import os
import shutil

import numpy as np
import scipy.linalg
import scipy.integrate as integrate
from scipy.interpolate import interp1d
from scipy.interpolate import RectBivariateSpline 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
# load important constants and parameters
# constants|
g = 3.721;               # gravitational acceleration at surface (m/s)

# material properties (assuming a basaltic lithology e.g. Abramov et al. (2016))
mats = {"Cp": 800.,      # heat capacity (J k^-1 C^-1) (Abramov et al. (2016))
        "K0": 19.3*1e9,  # adiabatic bulk modulus at zero pressure (GPa) (Abramov et al. (2016))
        "n": 5.5,        # derivative of the bulk modulus (Abramov et al. (2016))
        "rho": 3000.,    # uncompressed density (kg/m^3) (Abramov et al. (2016))
        "Dq":10000.,        # simple-to-complex crater transition diameter
        "Tliq":1250.,    # liquidus temperature (C) (Abramov et al. (2016))
        "Tsol":1100.,    # solidus temperature (C) (Abramov et al. (2016))
        "Tconv":1175,    # rheological transition temperature (Kolzenburg et al. (2018), Fig. 2)
        "gradT":13.,     # thermal gradient (C km^-1) (Babeyko & Zharkov (2000))
        "flux":35.       # (Plesa et al. (2015))
        }

# impact defaults
imdefs = {"vi"  :   10,     # impact velocity (km/s)
          "theta":  np.pi/4 # impact angle (radians)
        }

# material properties for pre-impact thermal profiles
bgmats = {
    'depth':np.array([1,50.,100.,3000])*1000,           # depth (m)
    'k': np.array([2.,3.,4.,100.]),                     # thermal conductivity (W/m/K)
    'Cp': np.array([800.,800.,1142.,1142.]),            # specific heat capacity (J/K/kg) (Szurgot, 2012)
    'rho':np.array([1800.,3000.0, 3500.0,3500.])        # density (kg/m^3)
}

# important stem locations
bms1 = 'BaseMeshes/'
cfgtemplate1 = 'config_template.cfg'

## Pre-impact thermal profiles

In [1]:
## pre-impact thermal profile functions

# pre-impact thermal profile
def pitp(t):return

def pitp_fromfile(filepath,xv,zv):
    # load temperatures
    x0grid, z0grid, Teq = np.load(filepath)
    
    if x0grid[0] == xv and z0grid[:,0] == zv:
        return Teq
    else:
        # interpolate onto new x and z grids
        Teq_intrp = RectBivariateSpline(x0grid[0],z0grid[:,0],Teq)

        # evaluate over new coordinates
        return Teq_interp(xv,zv)
    
    

# pre-impact thermal profiles (matching Abramov & Mojzsis (2016))
def pitp_cold_basic(depth, Tsurf=210., grad=13):
    return Tsurf + grad*depth/1000

def pitp_warm_basic(depth, Tsurf=274, grad=13):
    return Tsurf + grad*depth/1000


def pitp_cold(depth, Tsurf=210):
    #T = np.zeros(depth.shape)
    
    zlist = np.arange(0,3000000.,0.5)
    Tlist = np.zeros(zlist.shape)
    
    grad = mats['flux']/bgmats['k']
    
    Tlist[zlist<=bgmats['depth'][0]] = Tsurf + zlist[zlist<=bgmats['depth'][0]]*grad[0]/1000
    
    Tthresh = bgmats['depth'][0]*grad[0]/1000+Tsurf
    for i in range(1,len(bgmats['depth'])):
        
        inds = np.logical_and(zlist<=bgmats['depth'][i], zlist>bgmats['depth'][i-1])
        d0 = bgmats['depth'][i-1]
        
        Tlist[inds] = Tthresh+ (zlist[inds]-d0)*grad[i]/1000
        
        Tthresh += (bgmats['depth'][i]-d0)*grad[i]/1000
        
    Tint = interp1d(zlist, Tlist)
    return Tint(depth)


def pitp_warm(depth, Tsurf=274):
    #T = np.zeros(depth.shape)
    
    zlist = np.arange(0,3000000.,0.5)
    Tlist = np.zeros(zlist.shape)
    
    grad = mats['flux']/bgmats['k']
    
    Tlist[zlist<=bgmats['depth'][0]] = Tsurf + zlist[zlist<=bgmats['depth'][0]]*grad[0]/1000
    
    Tthresh = bgmats['depth'][0]*grad[0]/1000+Tsurf
    for i in range(1,len(bgmats['depth'])):
        
        inds = np.logical_and(zlist<=bgmats['depth'][i], zlist>bgmats['depth'][i-1])
        d0 = bgmats['depth'][i-1]
        
        Tlist[inds] = Tthresh+ (zlist[inds]-d0)*grad[i]/1000
        
        Tthresh += (bgmats['depth'][i]-d0)*grad[i]/1000
        
    Tint = interp1d(zlist, Tlist)
    return Tint(depth)


## Shock heating function

In [15]:
def shockT(depth,dist,Dtc,pitp,Tsurf,plot=0,vi=imdefs["vi"],theta=imdefs["theta"],n=mats["n"],K0=mats["K0"],Cp=mats["Cp"],rhot=mats["rho"],Dq = mats["Dq"]):
    '''
    Calculate post-impact shock heating using equations from Abramov et al. (2013) 
    inputs
    --------------
    depth: 
    dist:
    Dtr: transient crater diameter (km)

    parameters
    --------------
    vi: impact velocity (km/s)
    theta: impact angle (degrees)
    K0: adiabatic bulk modulus
    Cp: heat capacity (J k^-1 C^-1)
    rhot: target density
    '''
    Dtc=Dtc*1000
    Dtr=Dtc*1.2
    depth = depth*1000
    dist = dist*1000

    # projectile radius
    Dim = ((Dtc/1.16)*(vi*1000)**(-0.44)*g**(0.22)*np.sin(theta)**(-1/3))**(1/0.78)
    Rp = Dim/2
    
    print('projectile radius: ', Rp)

    # make r array
    r = np.sqrt((depth-Rp)**2+dist**2)

    # specific uncompressed target volume
    V0 = 1/rhot

    # pressure decay exponent
    k = 0.625*np.log10(vi)+1.25

    # pressure at r = Rp
    A = rhot*(vi*1000)**2/4 *np.sin(theta)

    # peak pressure
    P = A*(r/Rp)**(-k)
    P[r<Rp] = 0

    # waste heat
    dEw = 0.5*(P*V0-(2*K0*V0)/n)*(1-(P*n/K0+1)**(-1/n))+(K0*V0/(n*(1-n)))*(1-(P*n/K0+1)**(1-(1/n)))
    
    # temperature
    Tshock = dEw/Cp
    Tshock[Tshock<0] = 0

    # remove excavated material
    excinds = (dist)**2*(Dtr+4*Rp)/(Dtc**2)-0.25*(Dtr) - Rp + depth >= 0
    t2 =np.where(excinds[1:,:]!=excinds[0:-1,:])
    rollinds = [x for _, x in sorted(zip(t2[1],t2[0]))] 
    
    Tsadj = np.copy(Tshock)
    for i in [x for x,_ in sorted(zip(t2[1],t2[0]))]:
        Tsadj[:,i] = np.roll(Tshock[:,i],-rollinds[i]-1)
        Tsadj[-rollinds[i]-1:,i] = 0

    # central uplift
    Df = 0.91*Dtr**(1.125)/Dq**(0.09)
    Rcp = 0.22*(Df)/2
    cudis = 0.06*(Df/1000)**(1.1)*1000

    regul = depth[(dist<=Rcp)*(depth<=1.25*(0.25*Dtr))]
    regulr = dist[(dist<=Rcp)*(depth<=1.25*(0.25*Dtr))]
    
    dtemp = np.zeros(np.shape(Tshock))
    normr = np.max((regulr-Rcp)**2)
    normd = np.max(1.25*(0.25*Dtr)-regul)
    dtemp[(dist<=Rcp)*(depth<=(1.25*(0.25*Dtr)))] = cudis*(regulr-Rcp)**2*((1.25*(0.25*Dtr))-regul)/(normr*normd)
    hhh=depth+dtemp
    Tul = pitp(depth+dtemp)

    # plotting :)
    if plot:
        fig = plt.figure(figsize=(12,8))
        ax = fig.add_subplot(111)
        plt.contourf(dist/1000,depth/1000,Tul + Tshock,[20,45,70,95,120,200,500],cmap='YlOrRd')
        plt.colorbar();
        plt.contour(dist/1000,depth/1000,(dist)**2*(Dtr+4*Rp)/(Dtc**2)-0.25*(Dtr) - Rp + depth,[0])
        ax.set_aspect('equal')
        ax.invert_yaxis()

        fig = plt.figure(figsize=(12,8))
        ax = fig.add_subplot(111)
        plt.contourf(dist/1000,depth/1000,Tul + Tsadj,np.linspace(0,200,40),cmap='YlOrRd')
        plt.colorbar();
        ax.set_aspect('equal')
        ax.invert_yaxis()
        
    # make temperature dictionary
    Tout = {};
    
    Tout['Tul'] = Tul 
    Tout['Tsh'] = Tshock 
    Tout['Tshadj'] = Tsadj 
    Tout['Ttot'] = Tul + Tsadj 
    
    # make post-convective profile    
    Tsol = 1350+depth*(1850-1350)/500000
    Tliq = 1500+depth*(1965-1500)/500000

    Tthresh = 0.6*Tsol + 0.4*Tliq 
    Tthresh = np.maximum(Tthresh,pitp(depth,Tsurf=Tsurf))
    
    Tout['Tthresh'] = Tthresh
    
    Tout['Tconvadj'] = np.minimum(Tul + Tsadj, Tthresh)
    
    return Tout

## Basin class

In [16]:
## basin class
class Basin:
    def __init__(self,bname,Df):
        self.name = bname           # basin name
        self.Df = Df  # final diameter
        
        # get the rest of the important parameters
        self.getparams()
        
        return
        
    def doallT(self,xvals,zvals,pitp,savefp,bmstem=bms1,cfgtemp=cfgtemplate1,**shockkw):
        
        # make save dir if necessary
        if not os.path.isdir(savefp):
            os.mkdir(savefp)
        
        # set surface temperature
        self.Tsurf = pitp(0)
            
        # make temperature field
        print('Making post-impact T field')
        Tout = self.makeTfield(pitp,xvals,zvals,**shockkw)
        Tmat = Tout['Tconvadj']
        
        # save x, z, and T files
        xfp,zfp,Tfp = self.makexzT(savefp,xvals*1000,-zvals*1000,Tmat)
        
        # choose appropriate mesh file
        meshlims = np.array([[5,10,50,100,250,500],[10,50,100,250,500,800]])
        meshinds = meshlims[:,(meshlims[0,:]<=self.Dtc)*(meshlims[1,:]>self.Dtc)]

        basemeshfp = bmstem + str(meshinds[0,0]) + '_' + str(meshinds[1,0]) + 'km_rcm.inp'

        print('Using mesh file ' + str(meshinds[0,0]) + '_' + str(meshinds[1,0]) + 'km_rcm.inp')
        
        # modify and save new mesh 
        newmeshfp = savefp + str(meshinds[0,0]) + '_' + str(meshinds[1,0]) + 'km_rcm.inp'
        newmeshdf,xright,zbottom = self.makemesh(basemeshfp,savefp)
        
        # make config file
        self.makeconfig(cfgtemp,savefp,xfp,zfp,Tfp,newmeshfp,xright,zbottom)
        
        # make output file
        outputfp = savefp + 'output';
        if not os.path.isdir(outputfp):
            os.mkdir(outputfp)
        
        return 
    
    def getparams(self):
        '''
        Calculate other important crater parameters.
        
        Dtr: transient crater diameter
        cudis: central uplift displacement
        Rim: estimated impactor radius
        '''
        
        self.Dtc = ((1/0.91)*mats["Dq"]**(0.09)*(self.Df*1000))**(1/1.125)/1.2/1000   # Abramov & Kring (2005)
        self.cudis = 0.06*(self.Df)**(1.1)                         # Grieve et al. (1981)
        self.uplift = 1
        
        return
        
    def makeTfield(self,pitp,xvals,zvals,**kwargs):
        '''
        Make np array of initial temperature profile.
        
        pitp: pre-impact temperature profile (C)
        rdist: radial extent of domain (km)
        zdist: depth extent of domain (km)
        res: resolution of domain (km)
        '''
        # set surface temperature
        self.Tsurf = pitp(0)
        
        # initialize base position arrays
        dist,depth = np.meshgrid(xvals,zvals)
        
        # get shock temperature profile
        Tmat = shockT(depth,dist,self.Dtc,pitp,self.Tsurf,**kwargs)
        
        self.Tibounds = [np.max(dist),np.max(depth)]

        
        print('Made temperature field')
        return Tmat
   

    def makexzT(self,filepath,xvals,zvals,Tmat,app=''):
        '''
        Make x, z, and T files. You should have already run makeTfield over a domain
        as large or larger than the x-z space used here.
        '''
        xfp = filepath + r'xc' + app + '.txt'
        zfp = filepath + r'zc' + app + '.txt'
        Tfp = filepath + 'T_smooth' + app + '.txt'
        Teqfp = filepath + 'T_eq_smooth' + app + '.txt'

        # write x and z files
        np.savetxt(xfp,xvals.reshape(-1,1),fmt='%6.5f')
        np.savetxt(zfp,zvals.reshape(-1,1)[::-1],fmt='%6.5f')
        print('Wrote x to ' + xfp)
        print('Wrote z to ' + zfp)
        
        # write T file
        np.savetxt(Tfp,Tmat[::-1],fmt='%.2f')
        print('Wrote T to ' + Tfp)
        
        # write equilibrium T file
        T_eqp = np.tile(Tmat[:,-1],(len(xvals),1)).T
        np.savetxt(Teqfp,T_eqp[::-1])
        
        return xfp, zfp, Tfp
    
    
    def makemesh(self,basemeshfp,savefp):
        '''
        Load mesh from file and:
        1. Add a melt sheet or ejecta blanket above the crater if desired
        2. Save mesh grid locations to mesh_params.txt file
        '''
        # load in mesh
        dfbase = pd.read_csv(basemeshfp, sep=" ",names=[1,2,3,4,5,6,7,8,9,10,11], skiprows=1, engine='python');
        
        sec1 = dfbase[dfbase[3]!='quad'].astype('float')
        
        shutil.copy2(basemeshfp,savefp + 'mesh.inp')
        shutil.copy2(basemeshfp[:-4]+'.svg',savefp + 'mesh.svg')
        print(basemeshfp[:-4]+'.svg')
            
        # find unique indices 
        xun = np.sort(sec1[3].unique())
        zun = np.sort(sec1[4].unique())[::-1]
        
        xunm, zunm = np.meshgrid(xun,zun)
        meshparams = np.concatenate((xunm.flatten().reshape(-1,1),zunm.flatten().reshape(-1,1)),axis=1)
        np.savetxt(savefp+'mesh_params.txt', meshparams)
            
        # get locations of right and bottom boundaries
        xright = sec1[3].max()
        zbottom = dfbase[4].min()
        
        return dfbase, xright, zbottom
    
    
    def makeconfig(self,templatefp,savefp,xfp,zfp,Tfp,meshfp,xright,zbottom):
        # make copy of config template
        cfgsave = savefp + 'config.cfg'
        shutil.copy(templatefp,cfgsave) 
        
        cfg = open(cfgsave, "r")

        list_of_lines = cfg.readlines()
        
        # set file paths
        list_of_lines[3] = 'mesh_filename = "' + savefp + 'mesh.inp";\n';
        list_of_lines[4] = 'output_folder = "' + savefp + 'output\"' + ';\n';
        list_of_lines[5] = 'x_file        = "' + xfp + '";\n';
        list_of_lines[6] = 'z_file        = "' + zfp + '";\n';
        list_of_lines[7] = 'temp_file     = "' + Tfp + '";\n';
        list_of_lines[8] = 'eq_temp_file     = "' + savefp + 'T_eq_smooth.txt' + '";\n';
        
        # set surface temperature
        list_of_lines[17] = f'    T_surf          = {self.Tsurf:1.1f};             // double, surface temperature\n'

        # set material ids (gmsh cannot have 0 for material ids, but mars_heat treats them as indices)
        list_of_lines[29] = '    material_id                 = [1, 2, 3, 4]; // regolith, crust, lithosphere, mantle\n'
        
        # set mesh boundaries
        list_of_lines[73] = '    x_right  = %.1f;  // double\n'%xright;
        list_of_lines[74] = '    z_bottom = %.1f; // double\n'%zbottom;
        
        # save new config file
        cfg = open(cfgsave, "w")

        cfg.writelines(list_of_lines)
        cfg.close()
        
        return
        

## Workspace

In [21]:
xv100 = np.arange(0,1400.1,1.)
zv100 = np.arange(0,1200.1,1.)

n =800
d100km = Basin('%d km diameter'%n,n)
dfbase=d100km.doallT(xv100,zv100,pitp_cold,r'/media/sf_VB/Basins/iSale')

Making post-impact T field
(601, 801)
Made temperature field
Wrote x to /media/sf_VB/Basins/iSalexc.txt
Wrote z to /media/sf_VB/Basins/iSalezc.txt
Wrote T to /media/sf_VB/Basins/iSaleT_smooth.txt
Using mesh file 250_500km_rcm.inp
/home/sarahcate98/Sarah/BasinCooling/BaseMeshes/250_500km_rcm.svg
